In [54]:
# imports
import numpy as np
import re
import json

In [95]:
# inputs
test_data = open('input-test.txt').read().split('\n')
data = open('input.txt').read()

# Part 1

In [110]:
# test
content = test_data

def hex2bin(hex_string):
    binary = [bin(int(char, 16))[2:] for char in hex_string]
    binary = [(4-len(char)) * '0' + char for char in binary]
    return ''.join(binary)

def sum_versions(binary, version_sum = 0):

    if binary == '' or set(list(binary)) == {'0'}:
        return binary, version_sum
    
    remainder = None
    version = int(binary[:3], 2)
    type_id = int(binary[3:6], 2)
    data = binary[6:]
    version_sum += version

    if type_id == 4:
        groups = []
        chunks = [data[i:i + 5] for i in range(0, len(data), 5)]
        while len(chunks) > 0:
            if chunks[0][0] == '1':
                groups.append(chunks.pop(0)[1:])
            else:
                groups.append(chunks.pop(0)[1:])
                break
        literal = int(''.join(groups), 2)
        remainder = ''.join(chunks)
    else:
        length_id, data = data[0], data[1:]
        if length_id == '0':
            subpacket_length, data = data[:15], data[15:]
            subpacket_length = int(subpacket_length, 2)
            subpacket_content, remainder = data[:subpacket_length], data[subpacket_length:]
            while not (subpacket_content == '' or set(list(subpacket_content)) == {'0'}): 
                subpacket_content, version_sum = sum_versions(subpacket_content, version_sum)
            remainder = subpacket_content + remainder
        else:
            num_subpackets, remainder = data[:11], data[11:] 
            num_subpackets = int(num_subpackets, 2)
            count = 0
            while count < num_subpackets:
                remainder, version_sum = sum_versions(remainder, version_sum)
                count += 1

    return remainder, version_sum


version_sums = [6, 9, 14, 16, 12, 23, 31]
for idx, (c, s) in enumerate(zip(content, version_sums)):
    assert sum_versions(hex2bin(c))[-1] == s

In [111]:
# real
content = data
sum_versions(hex2bin(content))[-1]

953

# Part 2

In [119]:
# inputs
test_data = open('input-test-pt2.txt').read().split('\n')
data = open('input.txt').read()

In [122]:
# test
content = test_data

def eval_bits(binary, expression_eval = None):

    if binary == '' or set(list(binary)) == {'0'}:
        return binary, expression_eval
    
    remainder = None
    type_id = int(binary[3:6], 2)
    data = binary[6:]

    if type_id == 4:
        groups = []
        chunks = [data[i:i + 5] for i in range(0, len(data), 5)]
        while len(chunks) > 0:
            if chunks[0][0] == '1':
                groups.append(chunks.pop(0)[1:])
            else:
                groups.append(chunks.pop(0)[1:])
                break
        literal = int(''.join(groups), 2)
        remainder = ''.join(chunks)
        return remainder, literal
    else:
        values = []
        length_id, data = data[0], data[1:]
        if length_id == '0':
            subpacket_length, data = data[:15], data[15:]
            subpacket_length = int(subpacket_length, 2)
            subpacket_content, remainder = data[:subpacket_length], data[subpacket_length:]
            while not (subpacket_content == '' or set(list(subpacket_content)) == {'0'}): 
                subpacket_content, value = eval_bits(subpacket_content, expression_eval)
                values.append(value)
            remainder = subpacket_content + remainder
        else:
            num_subpackets, remainder = data[:11], data[11:] 
            num_subpackets = int(num_subpackets, 2)
            count = 0
            while count < num_subpackets:
                remainder, value = eval_bits(remainder, expression_eval)
                values.append(value)
                count += 1

        if type_id == 0:
            expression_eval = np.sum(values)
        elif type_id == 1:
            expression_eval = np.prod(values)
        elif type_id == 2:
            expression_eval = np.min(values)
        elif type_id == 3:
            expression_eval = np.max(values)
        elif type_id == 5:
            expression_eval = 1 if values[0] > values[1] else 0
        elif type_id == 6:
            expression_eval = 1 if values[0] < values[1] else 0
        elif type_id == 7:
            expression_eval = 1 if values[0] == values[1] else 0

    return remainder, expression_eval


values = [3, 54, 7, 9, 1, 0, 0, 1]
for idx, (c, v) in enumerate(zip(content, values)):
    assert eval_bits(hex2bin(c))[-1] == v

In [124]:
# real
content = data
eval_bits(hex2bin(content))[-1]

246225449979